###First, Load input data in form(x,y,z,x/y_perm,z_perm)

In [20]:
import xlrd
import math

x_perm = []
y_perm = []
z_perm = []

#load data
book = xlrd.open_workbook('data2.xlsx')
sheet = book.sheet_by_name('Sheet1')
data =[[sheet.cell_value(r,c) for c in range(2)] for r in range(60)]
y_perm2 = []
for i in range(len(data)):
    y_perm.append(data[i][0])
    x_perm.append(data[i][0])
    z_perm.append(data[i][1])

###Then create cells list - which consists of a dictionary for each cell, {x,y,z,x_perm,y_perm, z_perm}

In [9]:
cells=[]

#create cell dictionaries
for z in range(1,4): 
    for y in range(1,5):
        for x in range(1,6):
            mydict = {}
            mydict['y']=y
            mydict['x']=x
            mydict['z']=z
            cells.append(mydict) 
            
#populate dictionaries with data

for i in range(len(cells)):

    cells[i]['y_perm']=y_perm[i]
    cells[i]['x_perm']=x_perm[i]
    cells[i]['z_perm']=z_perm[i]

###Create method for airthmetic average of a list of cells or a list of values

In [4]:
def arithmetic(cells,k_to_avg):
    #it should take a list of cells and find the arithmetic average
    length = len(cells)
    if (type(cells[0])==dict):
        a = reduce(lambda x,y: x+y,(map(lambda x: x[k_to_avg] ,cells)))
    else:
        a = reduce(lambda x,y: x+y, map(lambda item: item ,cells))
        
    return(a/length)

###Create method for harmonic average of a list of cells or a list of values

In [5]:
def harmonic(cells,k_to_avg):
    #it should take a list of cells or list of values and find the harmonic average
    length = len(cells)
    
    if (type(cells[0])==dict):
        a = reduce(lambda x,y: x+y, map(lambda item: 1/(item[k_to_avg]) ,cells))
    else:
        a = reduce(lambda x,y: x+y, map(lambda item: 1/(item) ,cells))
    
    return(length/a)

###Create method for geometric average of a list of cells or a list of values

In [6]:
def geometric(cells,k_to_avg):
    #it should take a list of cells and take the geometric average
    length = len(cells)
    if(type(cells[0])==dict):
        a = reduce(lambda x,y: x*y,(map(lambda x: x[k_to_avg] ,cells)))
    else:
        a = reduce(lambda x,y: x*y,(map(lambda x: x ,cells)))
    return(a**(1.0/length))

###Calculate arithmetic averages for Kx, Ky and Kz

In [10]:
print(arithmetic(cells,'x_perm'))
print(arithmetic(cells,'y_perm'))
print(arithmetic(cells,'z_perm'))

141.133333333
141.133333333
4.78166666667


###Calculate harmonic averages for Kx, Ky and Kz

In [11]:
print(harmonic(cells,'x_perm'))
print(harmonic(cells,'y_perm'))
print(harmonic(cells,'z_perm'))

92.012156271
92.012156271
2.14895296487


###Calculate geometric averages for Kx, Ky and Kz

In [12]:
print(geometric(cells,'x_perm'))
print(geometric(cells,'y_perm'))
print(geometric(cells,'z_perm'))

112.010351441
112.010351441
3.04189797579


###Define the Carswell and Parsons Upper bound function

In [13]:
def carswell_upper(cells,k_to_avg,dim1,dim2,dim3):
    #dim1 is the fastest cycling dimension, then dim2 then dim3
    values = []
    results = []
    
    #find max dimensions of all dims
    d1_max = reduce(lambda x,y: x if x>y else y,map(lambda x: x[dim1] ,cells))
    d2_max = reduce(lambda x,y: x if x>y else y,map(lambda x: x[dim2] ,cells))
    d3_max = reduce(lambda x,y: x if x>y else y,map(lambda x: x[dim3] ,cells))
   
    for i in range(1,d3_max+1):
        for j in range(1,d2_max+1):
            for k in range(1,d1_max+1):
                #find cells that meet the current step
                values.append(filter(lambda cell: cell[dim3]== i and cell[dim2]==j and cell[dim1]==k,cells)[0])
        
        #calculate arithmetic mean of values collected at end of dim1 and dim2 cycles
        harms = arithmetic(map(lambda x: x,values),k_to_avg)
            
        #clear the temporary array and save the harmonic means
        values =[]
        results.append(harms)
    #calculate and return the harmonic mean of harmonic means
    return(harmonic(results,k_to_avg))

###Define Carswell and Parsons lower bounds function

In [14]:
def carswell_lower(cells,k_to_avg,dim1,dim2,dim3):
    #dim1 is the fastest cycling dimension, then dim2 then dim3
    values = []
    results = []
    
    #find max dimensions of all dims
    d1_max = reduce(lambda x,y: x if x>y else y,map(lambda x: x[dim1] ,cells))
    d2_max = reduce(lambda x,y: x if x>y else y,map(lambda x: x[dim2] ,cells))
    d3_max = reduce(lambda x,y: x if x>y else y,map(lambda x: x[dim3] ,cells))
   
    for i in range(1,d3_max+1):
        for j in range(1,d2_max+1):
            for k in range(1,d1_max+1):
                #find cells that meet the current step
                values.append(filter(lambda cell: cell[dim3]== i and cell[dim2]==j and cell[dim1]==k,cells)[0])
            
            #calculate harmonic mean of values collected at end of cycle in dim1
            harms = harmonic(map(lambda x: x,values),k_to_avg)
            #clear the temporary array and save the harmonic means
            values =[]
            results.append(harms)
    #calculate and return the arithmetic mean of the harmonic means
    return(arithmetic(results,k_to_avg))

###Calculate upper carswell and parsons bounds

In [15]:
print(carswell_upper(cells,'x_perm','y','z','x'))
print(carswell_upper(cells,'y_perm','x','z','y'))
print(carswell_upper(cells,'z_perm','x','y','z'))

128.642204826
125.917252914
2.83882035692


###Calculate lower carswell and parsons bounds

In [22]:
print(carswell_lower(cells,'x_perm','x','y','z'))
print(carswell_lower(cells,'y_perm','y','x','z'))
print(carswell_lower(cells,'z_perm','z','x','y'))

119.045514943
120.164955901
2.79792093858


###Calculate cardwell and parsons estimates

In [36]:
print 'k perm estimate: %f'%(math.sqrt(carswell_upper(cells,'x_perm','y','z','x')*carswell_lower(cells,'x_perm','x','y','z')))
print 'y perm estimate: %f' %(math.sqrt(carswell_upper(cells,'y_perm','x','z','y')*carswell_lower(cells,'y_perm','y','x','z')))
print 'z perm estimate: %f' %(math.sqrt(carswell_upper(cells,'z_perm','x','y','z')*carswell_lower(cells,'z_perm','z','x','y')))

k perm estimate: 123.750869
y perm estimate: 123.007484
z perm estimate: 2.818296


###Define Weiner upper bound (Kx, Ky, Kz) as Arithmetic mean

In [17]:
print(arithmetic(cells,'x_perm'))
print(arithmetic(cells,'y_perm'))
print(arithmetic(cells,'z_perm'))

141.133333333
141.133333333
4.78166666667


###Define Weinter lower bounds (Kx,Ky,Kz) as Harmonic means

In [18]:
print(harmonic(cells,'x_perm'))
print(harmonic(cells,'y_perm'))
print(harmonic(cells,'z_perm'))

92.012156271
92.012156271
2.14895296487
